# Aufgaben zum Tutorial - Grundlagen in TensorBoard

## Aufgabe 1

Starten Sie TensorBoard (Host: 0.0.0.0, Port: 6100, log_dir: /tf/tensorboard_log) aus einem Docker-Container über die Konsole und zeigen Sie die Weboberfläche in einem Browser an.


Tipp: Über den Befehl 

`docker exec -it CONTAINER_NAME /bin/bash`

können Sie die Bash-Konsole des Containers öffnen. Den Container-Namen erhalten Sie über den Befehl

`docker container ls`.

Öffnen der Docker-Konsole:

`docker exec -it CONTAINER_NAME /bin/bash`

Ausführen von TensorBoard:

`tensorboard --log_dir /tf/tensorboard_log --host 0.0.0.0 --port 6100`

## Aufgabe 2

Vervollständigen Sie das nachstehende Modell, sodass der Loss und die Metrik `accuracy` für jede Epoche im TensorBoard-Webinterface unter dem Tab `Scalars` angezeigt wird.

### Import der notwendigen Bibliotheken

In [ ]:
import os
import datetime

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import numpy as np
import matplotlib.pyplot as plt

### Import des MNIST Datensatzes

Dieser Datensatz besteht aus 28x28 Pixel Bildern, welche handgeschriebene Ziffern zwischen 0 und 9 darstellen. Jedes Bild besitzt ein Label mit der entsprechenden Zahl, welche auf dem Bild zu sehen ist. Dazu wird ein Sample des Datensatzes mit den Labels angezeigt.

In [ ]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

class_names = np.arange(10)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

### Erstellen des Keras-Modells

In [ ]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
    ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])  # Einfuegen von 'accuracy' als Metrik

### Training des Netzes 

In [ ]:
# Definition des log-Directory
log_dir = '/tf/tensorboard_log/run_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')  
# Erstellung des TensorBoard-Callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, write_graph=True)

model.fit(x=train_images[:1000],
          y=train_labels[:1000],
          epochs=50,
          validation_data=(test_images, test_labels),
          callbacks=[tensorboard_callback],  # Einfuegen des TensorBoard-Callbacks
          verbose=0)

## Aufgabe 3

Erweitern Sie das in Aufgabe 2 erstellte Keras-Modell, sodass eine zweite verdeckte Schicht (`Dense`) mit 128 Neuronen nach dem `Dense`-Layer eingefügt wird. Betrachten Sie den Modell-Graphen in TensorBoard. Welche Veränderungen zum Modell aus Aufgabe 2 können Sie feststellen?

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),  # Einfuegen des zweiten Dense-Layers
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])  # Einfuegen von 'accuracy' als Metrik

In [ ]:
# Definition des log-Directory
log_dir = '/tf/tensorboard_log/run_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')  
# Erstellung des TensorBoard-Callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, write_graph=True)

model.fit(x=train_images[:1000],
          y=train_labels[:1000],
          epochs=50,
          validation_data=(test_images, test_labels),
          callbacks=[tensorboard_callback],  # Einfügen des TensorBoard-Callbacks
          verbose=0)

## Aufgabe 4

Betrachten Sie den Verlauf der Loss- und Accuracy-Kurve von den Modellen aus Aufgabe 2 und Aufgabe 3. Was können Sie feststellen?

## Aufgabe 5

Vergleichen Sie die nachstehenden Parameterkonfigurationen für das Modell aus Aufgabe 3

- dropout_rate $\in$ [0.1, 0.2, 0.5, 0.8],
- Dense-Schicht 1 mit 32, 256 und 512 Neuronen und
- Dense-Schicht 2 mit 32, 256 und 512 Neuronen,

bezüglich der `accuracy`. Nutzen Sie dafür das Modul HParams in Tensorboard. 

Welche Parameter-Kombinationen sind ihrer Meinung nach die Besten?

Zusatz: Wie würden Sie die Zeit als Vergleichsparameter hinzufügen?

In [ ]:
# Festlegen der Parameter fuer hparams

HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([32, 256, 512]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([32, 256, 512]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.2, 0.5, 0.8]))
HP_TIME = hp.HParam('time', hp.RealInterval(0., 60.))

In [ ]:
def create_model(num_units_1, num_units_2, dropout_rate):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_units_1, activation=tf.nn.relu),
        tf.keras.layers.Dense(num_units_2, activation=tf.nn.relu),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    return model

def compile_and_fit(model, hparams, epochs, log_dir):
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_images[:1000],
        train_labels[:1000],
        epochs=epochs,
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir),
                   hp.KerasCallback(log_dir, hparams)],
        verbose=0
    )
    _, accuracy = model.evaluate(test_images, test_labels)
    return accuracy

In [ ]:
for units_1 in HP_NUM_UNITS_1.domain.values:
    for units_2 in HP_NUM_UNITS_2.domain.values:
        for dropout_rate in np.random.random_sample(3):
            hparams = {
                HP_NUM_UNITS_1: units_1,
                HP_NUM_UNITS_2: units_2,
                HP_DROPOUT: dropout_rate,
            }
            
            print('Running example with\nunits_1: {}\nunits_2: {}\ndropout_rate: {}'
                  .format(units_1, units_2, dropout_rate))
            
            log_dir = '/tf/tensorboard_log/hparam_t/run_' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
            
            model = create_model(units_1, units_2, dropout_rate)
            _ = compile_and_fit(model, hparams, 10, log_dir)

### Zusatz

Dies kann durch das Schreiben manueller Werte umgesetzt, also nicht über eine Keras-Callback, umgesetzt werden.